#50. 文区切り

(. or ; or : or ? or !) → 空白文字 → 英大文字というパターンを文の区切りと見なし，入力された文書を1行1文の形式で出力せよ．

In [99]:
import re #正規表現をインポート
from types import*

f_in = open("nlp.txt", "r") #ファイルを開けてreadlinesで全文を持ってくる
strings = f_in.read()

cut_p = "(.*?[.;:\?!])[ \\n]+([A-Z]|\\n)" #正規表現のパターンを作りコンパイル
#区切り文字で終わる任意の文字列(グループ０)、その後に空白か改行が１回以上続いて次に大文字か改行(グループ１)

re_cut_p = re.compile(cut_p)

tuple_list = re.findall(re_cut_p, strings)

lines = []
flag = 0

for elm in tuple_list:
    line = elm[0] + "\n"
    if ((line[0] == " ") | line[0].islower()): #小文字で始まるケースは前の分割で頭文字を持っていかれているのでそれを足す(空白も含めないと"A ~"をカバー出来ない)
        line = head + line
#章は文章と定義されていないので含めなかった（好み？）
    lines.append(line)
    head = elm[1]

for line in lines:
    print(line)


Natural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages.

As such, NLP is related to the area of humani-computer interaction.

Many challenges in NLP involve natural language understanding, that is, enabling computers to derive meaning from human or natural language input, and others involve natural language generation.

The history of NLP generally starts in the 1950s, although work can be found from earlier periods.

In 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence.

The Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sentences into English.

The authors claimed that within three or five years, machine translation would be a solved problem.

However, real progress was much slower, and after t

#51. 単語の切り出し


空白を単語の区切りとみなし，50の出力を入力として受け取り，1行1単語の形式で出力せよ．ただし，文の終端では空行を出力せよ．

In [100]:
word_list = []

for line in lines:
    words_of_line = line.split(" ") #空白文字で区切る
    for word in words_of_line: #区切った単語のリストを見ていく
        w_match = re.search("[.:;\?]", word) #文末文字があるかチェック
        new_word = re.sub("[\(\).,:;\n\?\-\"]", "", word) #記号を置換で削除する(stripとか使ったほうがスマート？)
        print(new_word)
        word_list.append(new_word)
        if w_match: #文末文字があったら改行挿入
            print("\n")
            word_list.append("\n")

Natural
language
processing
NLP
is
a
field
of
computer
science
artificial
intelligence
and
linguistics
concerned
with
the
interactions
between
computers
and
human
natural
languages


As
such
NLP
is
related
to
the
area
of
humanicomputer
interaction


Many
challenges
in
NLP
involve
natural
language
understanding
that
is
enabling
computers
to
derive
meaning
from
human
or
natural
language
input
and
others
involve
natural
language
generation


The
history
of
NLP
generally
starts
in
the
1950s
although
work
can
be
found
from
earlier
periods


In
1950
Alan
Turing
published
an
article
titled
Computing
Machinery
and
Intelligence
which
proposed
what
is
now
called
the
Turing
test
as
a
criterion
of
intelligence


The
Georgetown
experiment
in
1954
involved
fully
automatic
translation
of
more
than
sixty
Russian
sentences
into
English


The
authors
claimed
that
within
three
or
five
years
machine
translation
would
be
a
solved
problem


However
real
progress
was
much
slower
and
after
the
ALPAC
report
in

#52. ステミング
51の出力を入力として受け取り，Porterのステミングアルゴリズムを適用し，単語と語幹をタブ区切り形式で出力せよ． Pythonでは，Porterのステミングアルゴリズムの実装としてstemmingモジュールを利用するとよい．

ステミングとは、語形を標準型に戻す作業のこと。ポーターのアルゴリズムは、経験則に基いて段階を踏んで戻していく。

まずstemmingを使うための補足説明

In [101]:
##pip3 install nltk
from nltk.stem.porter import PorterStemmer
st = PorterStemmer()

st.stem("factionally")

'faction'

In [102]:
print("%20s \t %20s" %("単語", "語幹"))
for word in word_list:
    stemming = st.stem(word)
    print("%20s \t %20s" %(word, stemming))

                  単語 	                   語幹
             Natural 	                natur
            language 	              languag
          processing 	              process
                 NLP 	                  nlp
                  is 	                   is
                   a 	                    a
               field 	                field
                  of 	                   of
            computer 	               comput
             science 	               scienc
          artificial 	             artifici
        intelligence 	             intellig
                 and 	                  and
         linguistics 	             linguist
           concerned 	              concern
                with 	                 with
                 the 	                  the
        interactions 	             interact
             between 	              between
           computers 	               comput
                 and 	                  and
               human 	          

           attaching 	               attach
          realvalued 	             realvalu
             weights 	               weight
                  to 	                   to
                each 	                 each
               input 	                input
             feature 	               featur
                   
 	                    

                Such 	                 such
              models 	                model
                have 	                 have
                 the 	                  the
           advantage 	             advantag
                that 	                 that
                they 	                 they
                 can 	                  can
             express 	              express
                 the 	                  the
            relative 	                  rel
           certainty 	            certainti
                  of 	                   of
                many 	                 mani
           different 	          

#53. Tokenization

Stanford Core NLPを用い，入力テキストの解析結果をXML形式で得よ．また，このXMLファイルを読み込み，入力テキストを1行1単語の形式で出力せよ

In [103]:
#まずリンクからダウンロード
'''
git clone https://github.com/brendano/stanford_corenlp_pywrapper
cd stanford_corenlp_pywrapper
pip3 install .
'''
##mac, linux用なのでwindowsはどうなるかまだ調べてないです
#from stanford_corenlp_pywrapper import CoreNLP
##No module のエラーが出るので/usr/local/lib/python3.6/site-packages/stanford_corenlp_pywrapper/__init__.py
##を書き換える必要がある.

'\ngit clone https://github.com/brendano/stanford_corenlp_pywrapper\ncd stanford_corenlp_pywrapper\npip3 install .\n'

Line 1 of init.py, change "from sockwrap import *" to "from .sockwrap import *"

Line 194 of sockwrap.py, change "except socket.error, e" to "except socket.error as e"

Line 254 of sockwrap.py, change "except socket.timeout, e" to "except socket.timeout as e"

Lines 332 and 348, add parentheses around the print function argument.

352行も同様.

http://shirokai.hatenablog.com/entry/corenlp-python

In [104]:
##ここは今プログラム動かしてるところからみてダウンロードしたフォルダのがどこにあるか
#dir_name = "./stanford-corenlp-full-2016-10-31/*"
#proc = CoreNLP("pos", corenlp_jars=[dir_name])
#proc.parse_doc(strings)


するとpython3ではエラーを沢山吐いて動かない！corenlp-pythonの方で頑張ってみる

(pip3 install corenlp-python)

In [105]:
'''
import pprint
import json
import corenlp

parser = corenlp.StanfordCoreNLP(corenlp_path=dir_name)
result_json = json.loads(parser.parse(string))
pprint.pprint(result_json)
'''

'\nimport pprint\nimport json\nimport corenlp\n\nparser = corenlp.StanfordCoreNLP(corenlp_path=dir_name)\nresult_json = json.loads(parser.parse(string))\npprint.pprint(result_json)\n'

こっちもだめっぽい。というわけでjavaで直接やる必要がある。

最初に落としてきたstanford-corenlp-full-2016-10-31のフォルダで

sh corenlp.sh -annotators tokenize,ssplit,pos,lemma,ner,parse,dcoref -file ../nlp.txt

てな感じにやるとxmlで出力される

In [106]:
import xml.etree.ElementTree as ET
tree = ET.parse("nlp.txt.xml") #xmltreeの生成
root = tree.getroot() #rootの習得

for word in root.iter("word"): #再帰的にword属性を探しに行く
    print(word.text) #wordの中身を出力

Natural
language
processing
From
Wikipedia
,
the
free
encyclopedia
Natural
language
processing
-LRB-
NLP
-RRB-
is
a
field
of
computer
science
,
artificial
intelligence
,
and
linguistics
concerned
with
the
interactions
between
computers
and
human
-LRB-
natural
-RRB-
languages
.
As
such
,
NLP
is
related
to
the
area
of
humani-computer
interaction
.
Many
challenges
in
NLP
involve
natural
language
understanding
,
that
is
,
enabling
computers
to
derive
meaning
from
human
or
natural
language
input
,
and
others
involve
natural
language
generation
.
History
The
history
of
NLP
generally
starts
in
the
1950s
,
although
work
can
be
found
from
earlier
periods
.
In
1950
,
Alan
Turing
published
an
article
titled
``
Computing
Machinery
and
Intelligence
''
which
proposed
what
is
now
called
the
Turing
test
as
a
criterion
of
intelligence
.
The
Georgetown
experiment
in
1954
involved
fully
automatic
translation
of
more
than
sixty
Russian
sentences
into
English
.
The
authors
claimed
that
within
three
or
five

#54. 品詞タグ付け

Stanford Core NLPの解析結果XMLを読み込み，単語，レンマ，品詞をタブ区切り形式で出力せよ．

In [107]:
print("%20s \t %20s \t %20s" %("単語", "レンマ", "品詞"))

for token in root.iter("token"): #tokenを再帰的に探していく
    word = token.find("word").text #それぞれの属性を出力
    lemma = token.find("lemma").text
    pos = token.find("POS").text
    print("%20s \t %20s \t %20s" %(word, lemma, pos))

                  単語 	                  レンマ 	                   品詞
             Natural 	              natural 	                   JJ
            language 	             language 	                   NN
          processing 	           processing 	                   NN
                From 	                 from 	                   IN
           Wikipedia 	            Wikipedia 	                  NNP
                   , 	                    , 	                    ,
                 the 	                  the 	                   DT
                free 	                 free 	                   JJ
        encyclopedia 	         encyclopedia 	                   NN
             Natural 	              natural 	                   JJ
            language 	             language 	                   NN
          processing 	           processing 	                   NN
               -LRB- 	                -lrb- 	                -LRB-
                 NLP 	                  nlp 	                 

                 The 	                  the 	                   DT
            learning 	                learn 	                  VBG
          procedures 	            procedure 	                  NNS
                used 	                  use 	                  VBN
              during 	               during 	                   IN
             machine 	              machine 	                   NN
            learning 	             learning 	                   NN
       automatically 	        automatically 	                   RB
               focus 	                focus 	                   VB
                  on 	                   on 	                   IN
                 the 	                  the 	                   DT
                most 	                 most 	                  RBS
              common 	               common 	                   JJ
               cases 	                 case 	                  NNS
                   , 	                    , 	                 

#55. 固有表現抽出

入力文中の人名をすべて抜き出せ．

In [108]:
for token in root.iter("token"):
    ner = token.find("NER").text #NER　はname entity recognitionで場所や人名といった種別が入っている、人名はPERSONで入ってる
    if (ner == "PERSON"):
        print(token.find("word").text)

Alan
Turing
Joseph
Weizenbaum
MARGIE
Schank
Wilensky
Meehan
Lehnert
Carbonell
Lehnert
Racter
Jabberwacky
Moore


#56. 共参照解析

Stanford Core NLPの共参照解析の結果に基づき，文中の参照表現（mention）を代表参照表現（representative mention）に置換せよ．ただし，置換するときは，「代表参照表現（参照表現）」のように，元の参照表現が分かるように配慮せよ．

In [109]:
sentence_list = []
#単語を辞書形式でリスト化していき、そのリストを文章ごとにリスト化していく。
for sentences in root.iter("sentences"): #こうして階層化しないとcoreferenceあたりのsentenceも取ってきてしまう
    for sentence in sentences.iter("sentence"):
        word_list = []
        for token in sentence.iter("token"): #単語ごとに単語本体と単語の順番とを辞書化してリストにしたものを文章として収録していく
            word_txt = token.find("word").text #カッコの表記が単語に置き換えられているため、直す
            word_txt = re.sub(r'-LRB-',r'(',word_txt)
            word_txt = re.sub(r'-RRB-',r')',word_txt)
            word_dic = {"id":"", "text":""} #単語の順番と本体を収納する空の辞書を用意。
            word_dic["id"] = int(token.get("id"))
            word_dic["text"] = word_txt
            word_list.append(word_dic)
        sentence_list.append(word_list)   
    
coref_list = []

r_mention = ''

#共参照解析の結果の情報を辞書形式で収集する
for coreference in root.iter("coreference"):
    for mention in coreference.iter("mention"):
        #空の辞書を用意。(要素は代表参照表現、表現、行番号、開始単語位置、終了単語位置)
        coref_dic = {"r_mention":"", "mention":"", "sentence":"", "start":"", "end":""}
        if(mention.get("representative")): #mentionの表現の属性がrepresentativeだったら代表参照表現(一番最初にrepresentativeがくることを期待している)
            r_mention = mention.find("text").text
            r_mention = re.sub(r'-LRB-',r'(',r_mention) #先程と同様にカッコの表現を置換出直す
            r_mention = re.sub(r'-RRB-',r')',r_mention)
            
        mention_txt = mention.find("text").text
        mention_txt = re.sub(r'-LRB-',r'(',mention_txt) #こちらも同様
        mention_txt = re.sub(r'-RRB-',r')',mention_txt) 
        
        coref_dic["r_mention"] = r_mention #後で判定可能なので、代表参照表現の場合も同じ内容を入れる
        coref_dic["mention"] = mention_txt
        coref_dic["sentence"] = int(mention.find("sentence").text) #strで入っているのでintに直す
        coref_dic["start"] = int(mention.find("start").text)
        coref_dic["end"] = int(mention.find("end").text)
        coref_list.append(coref_dic)
        ###スタートとエンドとってその間単語を吐き出すのをやめる方式
        ##センテンスの中に単語のリストを作ることで対応

for coref_dic in coref_list: #共参照解析の辞書を一つずつ見ていく
    if (coref_dic["r_mention"] != coref_dic["mention"]): #もとから代表参照表現だったところは置換の必要がないため弾く
        loc = coref_dic["sentence"] - 1 #
        new_r_mention = "["+ coref_dic["r_mention"]+"]" + "<" + coref_dic["mention"] + ">" #代表参照（参照表現）の形に文章を整形。分かりやすいようカッコを変えた。
        for word_dic in sentence_list[loc]:
            if (coref_dic["start"] == word_dic["id"]):
                word_dic.update({"text":new_r_mention}) #mentionの開始位置の単語をnew_r_mentionに置き換える
            if ((coref_dic["start"] < word_dic["id"]) and (word_dic["id"] < coref_dic["end"])): #mentionの間の単語をnodataに置き換える
                word_dic.update({"text":"nodata"})

#新しく１行ずつの文章リストを作成する
new_sentence_list = []
for word_dic_list in sentence_list:
    new_sentence = ""
    for word_dic in word_dic_list:
        if (word_dic["text"] != "nodata"): #nodataは弾く
            new_sentence = new_sentence + " " + word_dic["text"] #空白を挟んで単語を連結していく
    new_sentence_list.append(new_sentence)

for new_sentence in new_sentence_list: #出力
    print(new_sentence)


 Natural language processing From Wikipedia , the free encyclopedia Natural language processing ( NLP ) is [the free encyclopedia Natural language processing ( NLP )]<a field of computer science> , artificial intelligence , and linguistics concerned with the interactions between computers and human ( natural ) languages .
 As such , NLP is related to the area of humani-computer interaction .
 Many challenges in NLP involve natural language understanding , that is , enabling computers to derive meaning from human or natural language input , and others involve natural language generation .
 History The history of NLP generally starts in the 1950s , although work can be found from earlier periods .
 In 1950 , Alan Turing published an article titled `` Computing Machinery and Intelligence '' which proposed what is now called the [Alan Turing]<Turing> test as a criterion of intelligence .
 The Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sent

#57. 係り受け解析
Stanford Core NLPの係り受け解析の結果（collapsed-dependencies）を有向グラフとして可視化せよ．

可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい

anacondaでgraphvizを落とす（brewだと拡張子に対応してない等のエラーが出てくる）:唯一の策

conda install -c conda-forge graphviz

pip3でも駄目

graphvizの方の問題らしい

python-graphvizを試してみる　も、駄目

In [110]:
import pydot
graph = [] #エッジのリストを一つのグラフとする
graph_list = [] #エッジのリストのリスト
for sentence in root.iter("sentence"): #文章ごとにとってくる
    for dependencies in sentence.iter("dependencies"): #文章の下の係り関係を取ってくる
        if (dependencies.get("type") == "collapsed-dependencies"):#collapsed-dependenciesのみ対象
            for dep in dependencies.iter("dep"): #governorがdependentに係る
                governor = dep.find("governor").text #かかり元
                dependent = dep.find("dependent").text #かかり先
                #pydotは,や.は受け付けない
                if (((governor != ".") and (governor != ",")) and((dependent != ".") and (dependent != ","))):
                    edge = [governor, dependent]
                    graph.append(edge)
    graph_list.append(graph)
    graph = []

fst_graph = graph_list[0]

g=pydot.graph_from_edges(fst_graph, directed=True)#リストをグラフにしてくれる
g.write_png('nlp.png', prog='dot')
print("Successfully done")

Successfully done


画像は第一文の係り受け解析結果。　線がだぶっているのは複数の種類の係り方があるから。

#58. タプルの抽出

Stanford Core NLPの係り受け解析の結果（collapsed-dependencies）に基づき，「主語 述語 目的語」の組をタブ区切り形式で出力せよ．

ただし，主語，述語，目的語の定義は以下を参考にせよ．

述語: nsubj関係とdobj関係の子（dependant）を持つ単語

主語: 述語からnsubj関係にある子（dependent）

目的語: 述語からdobj関係にある子（dependent）

In [111]:
nsubj_idxs_list = [] #各行ごとのnsubjのある位置のリスト
dobj_idxs_list = [] #各行ごとのdobjのある位置のリスト
sen_n_list = [] #それぞれの行のnsubjの辞書のリスト
sen_d_list = [] #dobjについても同様

for sentence in root.iter("sentence"): #行ごとに見てく
    nsubj_idxs = [] #行ごとにnsubjの位置を収録するリストの初期化
    dobj_idxs = [] #dobjについても同様
    n_list = [] #行ごとにnsubjの辞書を収録するリストの初期化
    d_list = [] #dobjについても同様
    for dependencies in sentence.iter("dependencies"): 
        if (dependencies.get("type") == "collapsed-dependencies"): #collapsed~以外を弾く
            for dep in dependencies.iter("dep"):
                nsubj_dec = {"gov_idx":"", "gov":"", "dep":""} #nsubjに使う辞書の初期化：governorの位置とgovnor本文とdependenceを取ってくる
                dobj_dec = {"gov_idx":"", "gov":"", "dep":""} #dobjについても同様
                if (dep.get("type") == "nsubj"): #depのtypeがnsubjの場合
                    nsubj_dec["gov_idx"] = dep.find("governor").get("idx") #nsubjの辞書を作成し、リストに追加
                    nsubj_idxs.append(nsubj_dec["gov_idx"]) #nsubjの位置のリストにも位置情報を別途追加
                    nsubj_dec["gov"] = dep.find("governor").text
                    nsubj_dec["dep"] = dep.find("dependent").text
                    n_list.append(nsubj_dec)
                if (dep.get("type") == "dobj"): #dobjの場合も同様
                    dobj_dec["gov_idx"] = dep.find("governor").get("idx")
                    dobj_idxs.append(dobj_dec["gov_idx"])
                    dobj_dec["gov"] = dep.find("governor").text
                    dobj_dec["dep"] = dep.find("dependent").text
                    d_list.append(dobj_dec)
    nsubj_idxs_list.append(nsubj_idxs) #行ごとにnsubj,dobjそれぞれ辞書のリストと位置のリストを追加していく
    dobj_idxs_list.append(dobj_idxs)
    sen_n_list.append(n_list)
    sen_d_list.append(d_list)
    
predicate_list = [] #述語,主語,目的語のリスト
subject_list = []
object_list = []

#zip関数により上で作成したリスト（nsubj,dobjそれぞれの辞書と位置のリスト計４本）について、各行ごとにリストを取り出していく
for (nsubj_idxs, dobj_idxs, n_list, d_list) in zip(nsubj_idxs_list, dobj_idxs_list, sen_n_list, sen_d_list):
    for n_gov_idx in nsubj_idxs: #ある行のnsubjの位置を見ていく
        if (n_gov_idx in dobj_idxs): #nsubjの位置がdobjのリストの中の位置と一致していたら
            pred_id = n_gov_idx #その単語を述語とみなし、位置を記録する
            for nsubj_dec in n_list: #以下、nsubjとdobjにおいて、述語のdependentを見つける
                if (nsubj_dec["gov_idx"] == pred_id):
                    predicate_list.append(nsubj_dec["gov"]) #nsubjの辞書のリストを見て、述語の位置にあるgovernorを取ってくる
                    subject_list.append(nsubj_dec["dep"]) #その述語のnsubjにおけるdependentは主語なので、それを取ってくる
            for dobj_dec in d_list:
                if (dobj_dec["gov_idx"] == pred_id):
                    object_list.append(dobj_dec["dep"]) #その述語のdobjにおけるdependentは目的語なので、それを取ってくる

print("%20s \t %20s \t %20s" %("主語", "述語", "目的語")) #出力
for (pred, subj, obj) in zip(predicate_list, subject_list, object_list):
    print("%20s \t %20s \t %20s" %(subj, pred, obj))

                  主語 	                   述語 	                  目的語
       understanding 	             enabling 	            computers
              others 	              involve 	           generation
              Turing 	            published 	              article
          experiment 	             involved 	          translation
               ELIZA 	             provided 	          interaction
             patient 	             exceeded 	                 base
               ELIZA 	              provide 	             response
               which 	           structured 	          information
       underpinnings 	          discouraged 	                 sort
                that 	            underlies 	             approach
                Some 	             produced 	              systems
               which 	                 make 	            decisions
             systems 	                 rely 	                which
                that 	             contains 	               er

#59. S式の解析

Stanford Core NLPの句構造解析の結果（S式）を読み込み，文中のすべての名詞句（NP）を表示せよ．入れ子になっている名詞句もすべて表示すること．

In [112]:
tokens_list = []
for sentence in root.iter("sentence"):
    for parse in sentence.iter("parse"): #parseに含まれるs式を取ってくる
        s_exp = parse.text
        s_exp = re.sub("\(", "( ", s_exp) #カッコの間に空白を入れて一括処理しやすくする
        s_exp = re.sub("\)" , " )", s_exp)
        tokens = s_exp.split(" ")
    tokens_list.append(tokens)

NP_list = []

for tokens in tokens_list:
    for i, token in enumerate(tokens):
        if (token == "NP"):
            depth = 1
            NP_chunk = tokens[(i + 1):] #NPが見えた時点で括弧が閉じるまでを一つの名詞句とする
            NP_tokens = []
            for i2, NP_token in enumerate(NP_chunk): #括弧が閉じるまで単語を読んでいく(左括弧で深さを＋１、右括弧で深さを−１)
                if (depth != 0):
                    if (NP_token == "("): #左括弧の横にはタグしかこないので無視
                        depth += 1
                    if (NP_token == ")"): #単語は右括弧の直前に来る
                        depth -= 1
                        if (NP_chunk[i2 - 1] != ")"): #右括弧が続く可能性があるので弾く
                            NP_tokens.append(NP_chunk[i2 - 1]) #NPに含まれるトークンを追加していく
            NP_list.append(NP_tokens) 

NP_words_list = []

for NP_tokens in NP_list: #名詞句のトークンリストを合体させて一つの単語にしていく
    NP_word = ""
    for NP_token in NP_tokens:
        if (NP_token == "-LRB-"): #括弧がLRBとRRBに置き換わっているので直す
            NP_token = "("
        if (NP_token == "-RRB-"):
            NP_token = ")"
        NP_word = NP_word + " " + NP_token #トークンを結合していく
    NP_words_list.append(NP_word)

for word in NP_words_list: #表示
    print(word)

 Natural language processing
 Wikipedia
 the free encyclopedia Natural language processing ( NLP )
 the free encyclopedia Natural language processing
 NLP
 a field of computer science , artificial intelligence , and linguistics concerned with the interactions between computers and human ( natural ) languages
 a field of computer science
 a field
 computer science
 artificial intelligence
 linguistics concerned with the interactions between computers and human ( natural ) languages
 linguistics
 the interactions between computers and human ( natural ) languages
 the interactions
 computers and human ( natural ) languages
 computers
 human ( natural ) languages
 such
 NLP
 the area of humani-computer interaction
 the area
 humani-computer interaction
 Many challenges in NLP
 Many challenges
 NLP
 natural language understanding , that is ,
 natural language understanding
 computers
 meaning
 human or natural language input
 others
 natural language generation
 History The history of NLP
 

 human language acquisition
 psycholinguistics
 NLL
 the related field of Computational Psycholinguistics
 the related field
 Computational Psycholinguistics
 the aims of computational language learning research
 the aims
 computational language learning research
 computational language
 research
 more
 human language acquisition , or psycholinguistics
 human language acquisition
 psycholinguistics
 NLL
 the related field of Computational Psycholinguistics
 the related field
 Computational Psycholinguistics
 the aims of computational language learning research
 the aims
 computational language learning research
 computational language
 research
 more
 human language acquisition , or psycholinguistics
 human language acquisition
 psycholinguistics
 NLL
 the related field of Computational Psycholinguistics
 the related field
 Computational Psycholinguistics
 the aims of computational language learning research
 the aims
 computational language learning research
 computational language
 r